In [4]:
import importlib

import preprocess_data
importlib.reload(preprocess_data)
from preprocess_data import *

import process_data_2
importlib.reload(process_data_2)
from process_data_2 import *

import constants2
importlib.reload(constants2)
from constants2 import *

import cnn_model
importlib.reload(cnn_model)
from cnn_model import create_cnn


HISTONE_TARGET = 'k36me3'
HELPERS = ['k27ac', 'k4me3']
CHROM_TRAIN = 'chr3'
N_TRAIN = 25
X_files = [DATA_PATH + 'OD8_k36me3.chr3.subs_train.b25.bedgraph', DATA_PATH + 'OD8_k27ac.chr3.subs_train.b25.bedgraph', DATA_PATH + 'OD8_k4me3.chr3.subs_train.b25.bedgraph']
y_file = DATA_PATH + 'OD8_k36me3.chr3.b25.bedgraph'

HISTONE_IMPL = 'k36me3'
HELPERS = ['k27ac', 'k4me3']
CHROM_IMPL = 'chr3'
BOUNDS_IMPL = {'start': 4600000, 'end': 4600010}
X_files_impl = [DATA_PATH + 'OD8_k36me3.chr3.subs_impl.b25.bedgraph', DATA_PATH + 'OD8_k27ac.chr3.subs_impl.b25.bedgraph', DATA_PATH + 'OD8_k4me3.chr3.subs_impl.b25.bedgraph']
y_file_impl = DATA_PATH + 'OD8_k36me3.chr3.b25.bedgraph'
  

def step_model_data(HISTONE_TARGET, HELPERS, CHROM_TRAIN, NAME_EXP, N_TRAIN):
    X_files, y_file = create_subs_data(HISTONE_TARGET, HELPERS, CHROM_TRAIN, NAME_EXP, subs_name = 'train', quality_percent = 0.2)
    print(X_files[0], SNR(X_files[0]))
    print(y_file, SNR(y_file))
    return X_files, y_file

def step_model(X_files, y_file, W, HISTONE_TARGET, HELPERS, CHROM_TRAIN, NAME_EXP, N_TRAIN):
    X_df, y_df = process_model_df(X_files, y_file, W, N_TRAIN)
    model = create_cnn(X_df, y_df , NAME_EXP+'.'+HISTONE_TARGET+'.'+CHROM_TRAIN+'.n_train_'+str(N_TRAIN)+'.h5')

    
def step_impl_data(HISTONE_IMPL, HELPERS, CHROM_IMPL, NAME_EXP):
    X_files_impl, y_file_impl = create_subs_data(HISTONE_IMPL, HELPERS, CHROM_IMPL, NAME_EXP, subs_name = 'impl', quality_percent = 0.2)
    print(X_files_impl[0], SNR(X_files_impl[0]))
    print(y_file_impl, SNR(y_file_impl))
    return X_files_impl, y_file_impl  
    
def step_impl(MODEL_F, HIST_IMPL_F, HELPERS_IMPL_F, CHROM_IMPL, HISTONE_IMPL, check_impl_f = None, bounds_impl = None):
    model_implementation(MODEL_F, HIST_IMPL_F, HELPERS_IMPL_F, CHROM_IMPL, HISTONE_IMPL, bounds_impl, check_impl_f)

def main(preprocessing = False):
    if preprocessing == True:
        X_files, y_file = step_model_data(HISTONE_TARGET, HELPERS, CHROM_TRAIN, NAME_EXP, N_TRAIN)
    step_model(X_files, y_file, W, HISTONE_TARGET, HELPERS, CHROM_TRAIN, NAME_EXP, N_TRAIN)
    MODEL_F = MODELS_PATH + NAME_EXP + '.'+HISTONE_TARGET+'.'+CHROM_TRAIN+'.n_train_'+str(N_TRAIN)+'.h5'
    if preprocessing == True:
        X_files_impl, y_file_impl = step_impl_data(HISTONE_IMPL, HELPERS, CHROM_IMPL, NAME_EXP)    
    step_impl(MODEL_F, X_files_impl[0], X_files_impl[1:], CHROM_IMPL, HISTONE_IMPL, check_impl_f = y_file_impl, bounds_impl = BOUNDS_IMPL)

main(preprocessing = False)